In [ ]:
#注意，应该先下载好pandas，numpy等
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import pmdarima as pm
from sklearn.preprocessing import StandardScaler

# 加载数据，读取到你的csv文件
#请注意，your_data是你的文件名
# df = pd.read_csv('your_data.csv')

# 数据预处理
# 示例：将日期时间分解为多个特征，你也可以用别的方法
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df['day'] = df['timestamp'].dt.day
df['month'] = df['timestamp'].dt.month
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['hour'] = df['timestamp'].dt.hour

# 标准化特征
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[['day', 'month', 'dayofweek', 'hour']])

# 构建随机森林模型
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(scaled_features, df['customer_count'])

# 构建ARIMA模型
arima_model = pm.auto_arima(df['customer_count'], seasonal=True, m=12)

# 模型融合
def ensemble_predict(arima_model, rf_model, features, n_periods):
    arima_forecast = arima_model.predict(n_periods=n_periods)
    rf_forecast = rf_model.predict(features[-n_periods:])
    return (arima_forecast + rf_forecast) / 2

# 预测未来值
n_periods = 30  # 预测未来30个时间点，你也可以进行更改。
future_features = scaler.transform(df[['day', 'month', 'dayofweek', 'hour']].tail(n_periods))
df['forecast'] = ensemble_predict(arima_model, rf_model, future_features, n_periods)
